# Execution Speed

In [1]:
from datetime import datetime
from src.imgdataconvertcodegen import get_execution_time

## Knowledge Graph Construction

### build from the scratch

In [2]:
from src.imgdataconvertcodegen import _builder

print(f'There are {len(_builder._edge_factories)} factory to use to construct the knowledge graph.')
begin_time = datetime.now()
_builder.build_from_scratch(_builder._edge_factories)
end_time = datetime.now()
print(_builder.knowledge_graph)
print(get_execution_time(begin_time, end_time))

There are 4 factory to use to construct the knowledge graph.
Knowledge Graph with 3960 nodes and 6402 edges.
The execution time is 0 days, 0 hours, 0 minutes, and 8.069446 seconds.


### load from the graph

In [3]:
begin_time = datetime.now()
_builder.build_from_file(_builder._know_graph_file_path)
end_time = datetime.now()
print(_builder.knowledge_graph)
print(get_execution_time(begin_time, end_time))

Knowledge Graph with 3960 nodes and 6402 edges.
The execution time is 0 days, 0 hours, 0 minutes, and 0.376994 seconds.


## Get Conversion Code with metadata as input

In [4]:
from src.imgdataconvertcodegen import get_conversion

start_time = datetime.now()
source_metadata = {
    "data_representation": "numpy.ndarray",
    "color_channel": "rgb",
    "channel_order": "channel last",
    "minibatch_input": False,
    "data_type": "uint8",
    "intensity_range": "full",
    "device": "cpu"
}
target_metadata = {
    "data_representation": "torch.tensor",
    "color_channel": "rgb",
    "channel_order": "channel first",
    "minibatch_input": True,
    "data_type": "uint8",
    "intensity_range": "full",
    "device": "cpu"
}
conversion = get_conversion("input", source_metadata, "output", target_metadata)
end_time = datetime.now()
print(f"The imports code is: \n{conversion[0]}\n")
print(f"The conversion code is: \n{conversion[1]}")
print(get_execution_time(start_time, end_time))

The imports code is: 
import torch

The conversion code is: 
var_474dfed778794785a1f5124d5b37f1f7 = torch.from_numpy(input)
var_7b06cb65d8994319b59a48267c2714a1 = var_474dfed778794785a1f5124d5b37f1f7.permute(2, 0, 1)
output = torch.unsqueeze(var_7b06cb65d8994319b59a48267c2714a1, 0)
The execution time is 0 days, 0 hours, 0 minutes, and 0.001995 seconds.


## Get Conversion Code with library name as input

In [5]:
# todo, meansure convert_code not in the cache
start_time = datetime.now()
conversion_using_lib_name = get_conversion("input", 'scikit-image', "output", 'torch', 'color', 'color')
end_time = datetime.now()
print(f"The imports code is: \n{conversion_using_lib_name[0]}\n")
print(f"The conversion code is: \n{conversion_using_lib_name[1]}")
print(get_execution_time(start_time, end_time))

The imports code is: 
import torch

The conversion code is: 
var_5480671bcd57441ab016343f0e771c13 = torch.from_numpy(input)
var_cd626dfb61ed4fe1a9ee4227c3bc0b88 = var_5480671bcd57441ab016343f0e771c13.permute(2, 0, 1)
output = torch.unsqueeze(var_cd626dfb61ed4fe1a9ee4227c3bc0b88, 0)
The execution time is 0 days, 0 hours, 0 minutes, and 0.0 seconds.


## Get Conversion Path

In [6]:
from src.imgdataconvertcodegen import get_convert_path
start_time = datetime.now()
path = get_convert_path(source_metadata, target_metadata)
end_time = datetime.now()
print("the path is:")
for metadata in path:
    print(f'\t{metadata}')
print(get_execution_time(start_time, end_time))

the path is:
	{'data_representation': 'numpy.ndarray', 'color_channel': 'rgb', 'channel_order': 'channel last', 'minibatch_input': False, 'data_type': 'uint8', 'intensity_range': 'full', 'device': 'cpu'}
	{'data_representation': 'torch.tensor', 'color_channel': 'rgb', 'channel_order': 'channel last', 'minibatch_input': False, 'data_type': 'uint8', 'intensity_range': 'full', 'device': 'cpu'}
	{'data_representation': 'torch.tensor', 'color_channel': 'rgb', 'channel_order': 'channel first', 'minibatch_input': False, 'data_type': 'uint8', 'intensity_range': 'full', 'device': 'cpu'}
	{'data_representation': 'torch.tensor', 'color_channel': 'rgb', 'channel_order': 'channel first', 'minibatch_input': True, 'data_type': 'uint8', 'intensity_range': 'full', 'device': 'cpu'}
The execution time is 0 days, 0 hours, 0 minutes, and 0.001994 seconds.


## Convert Image Data

In [7]:
import numpy as np
import torch

start_time = datetime.now()
source_image = np.random.randint(0, 256, (20, 20, 3), dtype=np.uint8)
expected_image = torch.from_numpy(source_image).unsqueeze(0).permute(0, 3, 1, 2)
code = get_conversion("source_image", source_metadata, "actual_image", target_metadata)
exec(code[1])
end_time = datetime.now()
print(get_execution_time(start_time, end_time))

The execution time is 0 days, 0 hours, 0 minutes, and 0.0 seconds.


## Add new Metadata spec

In [8]:
from src.imgdataconvertcodegen import add_image_metadata
start_time = datetime.now()
add_image_metadata({
    "data_representation": "PIL.Image",
    "color_channel": "rgb",
    "channel_order": "channel last",
    "minibatch_input": False,
    "data_type": "uint8",
    "intensity_range": "0to255",
    "device": "cpu"
})
end_time = datetime.now()
print(get_execution_time(start_time, end_time))

The execution time is 0 days, 0 hours, 0 minutes, and 0.072806 seconds.


## Add a new factory

In [9]:
from src.imgdataconvertcodegen import add_convert_code_factory

def convert_torch_to_numpy(source, target):
    if source["data_representation"] == "torch.tensor" and target["data_representation"] == "numpy.ndarray":
        return '', f"actual_image = source_image.numpy()"
    return None

start_time = datetime.now()
# the graph will be rebuilt using this new factory
add_convert_code_factory(convert_torch_to_numpy)
end_time = datetime.now()
print(get_execution_time(start_time, end_time))

The execution time is 0 days, 0 hours, 0 minutes, and 4.512945 seconds.


## Add a new librariy metadata preset

In [10]:
from src.imgdataconvertcodegen import add_new_lib_preset

start_time = datetime.now()
add_new_lib_preset("PIL.image", 'color', {
    "data_representation": "PIL.Image",
    "color_channel": "rgb",
    "channel_order": "channel last",
    "minibatch_input": False,
    "data_type": "uint8",
    "intensity_range": "0to255",
    "device": "cpu"
})
end_time = datetime.now()
print(get_execution_time(start_time, end_time))

The execution time is 0 days, 0 hours, 0 minutes, and 0.0 seconds.
